In [ ]:
import dataset
import importlib
import json
import numpy as np
import os
import reasoning
import torch
import qa
import time

from dataset.bgquiz import BGQuiz
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForQuestionAnswering, BertForMultipleChoice
from reasoning.multichoice import *
from tqdm import tqdm
from qa.utils import query_es

# Reload the cached modules
_ = importlib.reload(dataset.bgquiz)
_ = importlib.reload(qa.utils)
_ = importlib.reload(reasoning.multichoice)

In [ ]:
#RACE two epochs
model_dir = "models/SLAVIC_BERT_RACE3/"

# Example for a Bert model
model = BertForMultipleChoice.from_pretrained(model_dir, num_choices=4)
tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=False)  # Add specific options if needed
model.to('cuda')

In [ ]:
wiki_control = \
'''
Обсадата на Одрин приключва с превземането на турската крепост от Втора българска армия под общото командване на ген. Никола Иванов. Решителният пробив на източния сектор на крепостта е извършен под командването на ген. Георги Вазов. Ръководещият защитата на Одрин – Шукри паша, предава сабята си на ген. Иванов с думите: „Храбростта на българската армия е безподобна. На такава храброст никоя крепост не може да устои“. Победата фактически решава изхода на Балканската война.
'''

question_control = "Кои военачалници ръководят защитата и превземането на Одринската крепост през месец март 1913 г.?"
options_control = [
         "Абдулах паша – ген. Радко Димитриев",
         "Явер паша – ген. Васил Кутинчев",
         "Назим паша – ген. Данаил Николаев",
         "Шукри паша – ген. Никола Иванов"
      ]

question_control = \
'''
Името на коя международна организация е пропуснато в текста от нейния Устав, 
подписан през 1945 г. в Сан Франциско?    
„1. _____________________________ е основана върху принципа на пълното равенство на  своите членове. 
3. Те се задължават да уреждат международните си спорове с мирни  средства. 
4. Членовете се въздържат в отношенията си от заплаха или от употреба на  сила. 
7. Организацията не се намесва във вътрешните работи на държавите. 
Това обаче не  засяга прилагането на принудителни мерки в случай на заплаха срещу мира, 
на нарушение на  мира и на актове на агресия.”'''
wiki_control = '''
Никола Иванов (генерал)\n\nНикола Иванов Иванов е български офицер (генерал от пехотата), началник на Щаба на войската през 1894–1896 година и министър на войната (1896–1899). Командир на Втора българска армия през Балканските войни от 1912–1913 година. Ръководи обсадата и превземането на Одринската крепост (март 1913).\n\nНикола Иванов е роден на 2 март 1861 г. в Калофер. Учи в Априловската гимназия '''
options_control = [
         "Лигата на нациите",
         "Организацията на обединените нации",
         "Международната агенция за атомна енергия",
         "Международната организация на труда"
      ]


        
predict(model, tokenizer, wiki_control, question_control, options_control, use_gpu=True)

In [ ]:
es = Elasticsearch()
name = "bgwiki_paragraph"

In [ ]:
quiz = BGQuiz("data/bg_rc-v1.0.json")

In [ ]:
query_fields = [
#     ['title.bulgarian', 'passage.bulgarian'],
#     ['title.bulgarian', 'passage.ngram'],
#     ['title.bulgarian', 'passage.ngram', 'passage', 'passage.bulgarian'],
#     ['passage.ngram', 'passage', 'passage.bulgarian^2'],
    ['title.bulgarian^2', 'passage.ngram', 'passage', 'passage.bulgarian^2'],
]

for query_field in query_fields:
    #, 'bgwiki_window'
    for name in ['bgwiki_paragraph']:
        for cr in [1, 2, 5, 10, 20]:
            start = time.time()
            
            setup = dict()
            setup['results_count'] = cr
            setup['query_field'] = query_field.copy()
            setup['index'] = name
            setup['model'] = model_dir

            print(json.dumps(setup, indent=2))
            print()

            preds = {}

            for i, q in enumerate(quiz.iterator()):
                question = q.question
                answers = q.answers

                try:
                    passages = set()
                    for a in answers:
                        passages.update([
                            x['source']['passage'] for x in query_es(
                                es, question + ' ' + a, setup['index'],
                                setup['results_count'], setup['query_field'])
                        ])

                    ans_cnt = len(answers)
                    if ans_cnt < 4:
                        answers.append(' ')

                    votes = np.zeros(4, dtype=np.int32)
                    sc = np.ones((1, 4), dtype=np.float32)
                    for p in passages:
                        scores = predict(
                            model, tokenizer, p, question, answers, use_gpu=True)
                        if ans_cnt < 4:
                            scores[0, 3] = -np.inf
                        sc += softmax(scores)

                        votes[scores.argmax()] += 1

    #                 print(str(i + 1) + './' + str(len(quiz.data_gen)), question)
    #                 print(answers, q['correct'])
    #                 print(answers[sc.argmax()], votes)

                    preds[q.id] = answers[sc.argmax()]

                    #                 print()
                    if (i) % 100 == 0:
                        print(
                            str(i + 1) + './' + str(quiz.size()), end='...')
                except KeyboardInterrupt:
                    raise

            print()
            dirname = str(int(datetime.now().timestamp()))
            path = 'experiments/{}'.format(dirname)
           
            try:
                if not os.path.exists(path):
                    os.makedirs(path)
            except OSError:
                print("Creation of the directory %s failed" % path)
            else:
                print("Successfully created the directory %s " % path)

            with open(path + '/preds.json', 'w') as o:
                json.dump(preds, o, ensure_ascii=False)

            with open(path + '/setup.json', 'w') as o:
                json.dump(setup, o, ensure_ascii=False, indent=2)
                
            

            print(path)
            !python evaluation/evaluate.py data/bg_rc-v1.0.json $path/preds.json  --out-file $path/eval.json
            
            end = time.time()
            print("Done in {} seconds", int(end - start))